In [14]:
%run gpt_model.ipynb

In [15]:
with open('wizard.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(set(text))

with open("vocab.txt", "w", encoding="utf-8") as f:
    for ch in chars:
        f.write(ch + "\n")
        
vocab_size = len(chars)

In [16]:
char_to_int = {ch:i for i,ch in enumerate(chars)}
encode = lambda s: [char_to_int[c] for c in s]

data = torch.tensor(encode(text), dtype=torch.long)

In [17]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [18]:
model = GPTLanguageModel(vocab_size)
m = model.to(device)

In [19]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [20]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
# model.parameters() -> all trainable weights in the model
# Each parameter tensor has .data and .grad

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss {losses['train']:.3f}, val loss {losses['val']:.3f}")
    
    xb, yb = get_batch('train')
    
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True) # parameter.grad is None
    loss.backward()
    optimizer.step()
print(loss.item())

torch.save(m, "model")

step: 0, train loss 4.418, val loss 4.419
step: 250, train loss 3.162, val loss 3.213
step: 500, train loss 3.178, val loss 3.194
step: 750, train loss 3.150, val loss 3.195
3.4073634147644043
